In [ ]:
import QuantLib as ql

import csv
from typing import List, Tuple


def read_swap_quotes(date: ql.Date) -> List[Tuple[ql.Period, ql.RelinkableQuoteHandle]]:
    date_as_int = date.year() * 10000 + date.month() * 100 + date.dayOfMonth()
    file_path = 'data/swap_rates_' + str(date_as_int) + '.csv'
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        rates = [(ql.PeriodParser.parse(str(r[0])), float(r[1])) for r in reader]
        return [(q[0], ql.RelinkableQuoteHandle(ql.SimpleQuote(q[1]))) for q in rates]


def read_fixings() -> List[Tuple[ql.Date, float]]:
    file_path = 'data/fixings.csv'
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        return [(ql.Date(str(r[0]), '%Y%m%d'), float(r[1]) / 100.0) for r in reader]

In [ ]:
# USD LIBOR SWAP CONVENTIONS

SETTLEMENT_DAYS = 2
BUSINESS_CONVENTION = ql.Following
DAY_COUNT = ql.Thirty360()
CALENDAR = ql.UnitedKingdom(ql.UnitedKingdom.Settlement)
FXD_FREQUENCY = ql.Semiannual

TS_DAY_COUNT = ql.Actual365Fixed() # term structure day count

# USD LIBOR SWAP CURVE CONSTRUCTION FUNCTION

def build_swap_curve(valuation_date: ql.Date, 
                     index: ql.IborIndex, 
                     quote_handles: List[ql.QuoteHandle]) -> ql.YieldTermStructure:
    settlement = CALENDAR.advance(today, SETTLEMENT_DAYS, ql.Days)
    instruments = [ql.SwapRateHelper(q, t, CALENDAR, FXD_FREQUENCY, BUSINESS_CONVENTION, DAY_COUNT, index) 
                   for t, q in quote_handles]
    crv = ql.PiecewiseLogCubicDiscount(settlement, instruments, TS_DAY_COUNT)
    crv.enableExtrapolation()
    return crv

In [ ]:
# VALUATION DATE

today = CALENDAR.adjust(ql.Date(30, ql.June, 2021))
ql.Settings.instance().evaluationDate = today

# CREATE A TERM STRUCTURE HANDLE

ts_handle = ql.RelinkableYieldTermStructureHandle()

# CREATE INDEX AND STORE HISTORICAL FIXINGS

euribor_idx = ql.USDLibor(ql.Period(3, ql.Months), ts_handle)
dates, fixings = zip(*read_fixings())
euribor_idx.addFixings(dates, fixings)

# CONSTRUCT AN INTEREST RATE SWAP TERM STRUCTURE

swap_quotes = read_swap_quotes(today)
swap_ts = build_swap_curve(today, euribor_idx, swap_quotes)

# LINK SWAP TERM STRUCTURE TO A HANDLE

ts_handle.linkTo(swap_ts)